In [ ]:
import serial

In [ ]:
class RP7972A():
    """Esta clase pide y mide los datos de la fuente, para luego ser procesados"""
    def __init__(self, curr_initialFreq, curr_finalFreq, curr_amplitude, curr_sweepPoints):
        """Define variables de entrada como variables globales en la clase"""
        self.curr_initialFreq = curr_initialFreq
        self.curr_finalFreq = curr_finalFreq
        self.curr_amplitude = curr_amplitude
        self.curr_sweepPoints = curr_sweepPoints

        # FOR INITIALIZATION OF POWER SOURCE (RP7972A)
        #open serial port
        COM_PORT = "COM5"  # Instrument port location77
        TIMEOUT = 1
        self.USB_scpi = serial.Serial()
        self.USB_scpi.port=COM_PORT
        self.USB_scpi.timeout = TIMEOUT
        self.USB_scpi.open()

    def generate_arb(self, frequency):
        """Para generar las señales de corriente que se envian a la fuente, se tiene que llamar por cada frecuencia, no es sweep"""
        """INFO: service manual, cap 4, Programming an Arbitrary Waveform"""
        sin_points, dwell_time = self.arb_data(frequency)
        arb_cycles = self.cycles_per_frequency(frequency)
        # Enable arb function
        arb_enable = "CURR:MODE ARB"
        self.scpi_out(arb_enable)
        # Specify arb type
        arb_type = "ARB:FUNC:TYPE CURR"
        self.scpi_out(arb_type)
        # Specify dwell time
        arb_dwell = "ARB:CURR:CDW:DWEL " + dwell_time
        self.scpi_out(arb_dwell)
        # Program arb points
        arb_points = "ARB:CURR:CDW " + sin_points
        self.scpi_out(arb_points)
        # Speecify the arb to repeat
        if arb_cycles > 1:
            arb_repeat = "ARB:COUN " + str(arb_cycles)
            self.scpi_out(arb_repeat)
        # Specify the last output
        arb_end = "ARB:END:LAST OFF"
        self.scpi_out(arb_end)
        
    def measure_voltage():
        """Para configurar como tomar datos"""
        # TODO SIG
        
    # TOOLBOX
    def sweep_frequencies(self):
        """A partir de las frecuencias inicial y final y la cantidad de sweep points se definen las frecuencias sobre las que se hará el barrido"""
        #TODO escribir funcion con curr_initialFreq, curr_finalFreq, curr_sweepPoints
    def cycles_per_frequency(self, freq):
        """Switch para definir ciclos por frecuencia, recibe una frequencia saca cantidad de ciclos"""
        if freq < x1:
            cycle = y1
        elif x1 <= freq < x2:
            cycle = y2
        elif freq <= x3:
            cycle = y3
        # TODO SIG
        return cycles
    def arb_data(self, frequency):
        """Generates data points for arb generation and dwell time for a frequency that is input"""
        # TODO SIG
        return arb_points, dwell_time # arb_points and dwell_time must be strings
    def scpi_out(self, command):
        """Outputs an encoded scpi string on serial port"""
        while not self.USB_scpi.is_open:
            self.USB_scpi.close() #idk if this is needed for redundancy
            print("!!!Serial not open!!!")
            COM_PORT="COM" + input("enter new COM port: ")
            self.USB_scpi.port= COM_PORT
            self.USB_scpi.open()
 
        self.USB_scpi.write(str(command).encode())  # Send command


        
    

class HMI():
    """Esta clase es la interfaz entre el usuario y nuestro programa"""
    def __init__():


class EIS():
    """Una vez tengamos los datos, se procesaran con esta clase"""
    

In [ ]:
"""snippet for serial communication with the instrument"""
# validate_serial.py
import serial

COM_PORT = "COM5"  # Instrument port location77
TIMEOUT = 1
CHECK_COMMAND = "*IDN?\n"  # Terminate with newline

with serial.Serial(
  COM_PORT,
  write_timeout=TIMEOUT,
  timeout=TIMEOUT) as serial_connection:
    serial_connection.write(CHECK_COMMAND.encode())  # Send command
    response = serial_connection.readline().decode()  # read response
print(response)